In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Load training datasets and testing datasets 
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')


## Evaluate models that predict the categories of the attacks: 

In [3]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic'], 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [4]:
xcat, ycat = preprocessing(test_cat)
xcat_eq, ycat_eq = preprocessing(test_cat_eq)


In [5]:
# Evaluate models that predict category: FNR, F1, Precision, Recall, Accuracy, FP, FN, TP, TN
def evaluate_cat(model, x, y): 
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y, axis=1)

    report = classification_report(y_true, y_pred, output_dict=True)
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    accuracy = report['accuracy']

    cat = set(y_pred)
    true_cat = set(y_true)

    conf_matrix = confusion_matrix(y_true, y_pred)
    return "f1: "+str('%.3f'%f1), "precision: "+str('%.3f'%precision), "recall: "+str('%.3f'%recall), "accuracy: "+str('%.3f'%accuracy), "true categories: "+str(true_cat), "predicted categories: "+str(cat), conf_matrix
    

#### 302

In [19]:
# Load models 
model_glob = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id302.hdf5')
model_loc0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id302.hdf5')
model_loc1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id302.hdf5')
model_loc2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id302.hdf5')


In [20]:
evaluate_cat(model_glob, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.745',
 'precision: 0.896',
 'recall: 0.655',
 'accuracy: 0.655',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  154,     0,    22,   773],
        [  942,     0,   265,  1735],
        [17239,     0, 35675,   763],
        [  520,     0,    19,  6409]], dtype=int64))

In [21]:
evaluate_cat(model_loc0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.621',
 'precision: 0.896',
 'recall: 0.501',
 'accuracy: 0.501',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  153,     0,    21,   775],
        [ 1039,     0,   185,  1718],
        [27203,     0, 25707,   767],
        [  466,     0,    18,  6464]], dtype=int64))

In [22]:
evaluate_cat(model_loc1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.745',
 'precision: 0.896',
 'recall: 0.655',
 'accuracy: 0.655',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  152,     0,    22,   775],
        [  896,     0,   290,  1756],
        [17230,     0, 35677,   770],
        [  482,     0,    21,  6445]], dtype=int64))

In [23]:
evaluate_cat(model_loc2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.746',
 'precision: 0.896',
 'recall: 0.656',
 'accuracy: 0.656',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  141,     0,    25,   783],
        [  914,     0,   282,  1746],
        [17227,     0, 35677,   773],
        [  444,     0,    21,  6483]], dtype=int64))

#### 310

In [28]:
mod_310 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id310.hdf5')

In [29]:
evaluate_cat(mod_310, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.263',
 'precision: 0.873',
 'recall: 0.212',
 'accuracy: 0.212',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   36,     0,    38,   875],
        [  674,     0,     2,  2266],
        [44219,     0,  6840,  2618],
        [   45,     0,   103,  6800]], dtype=int64))

#### 311, 312, 313

In [31]:
mod_311 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id311.hdf5')
mod_312 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id312.hdf5')
mod_313 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id313.hdf5')

In [32]:
evaluate_cat(mod_311, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.812',
 'precision: 0.857',
 'recall: 0.806',
 'accuracy: 0.806',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    7,     0,   189,   753],
        [    0,     0,    37,  2905],
        [    2,     0, 46124,  7551],
        [    1,     0,  1058,  5889]], dtype=int64))

In [34]:
evaluate_cat(mod_312, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.075',
 'precision: 0.841',
 'recall: 0.135',
 'accuracy: 0.135',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    7,     0,     8,   934],
        [    3,     0,     1,  2938],
        [    8,     0,  1764, 51905],
        [   17,     0,     3,  6928]], dtype=int64))

In [36]:
evaluate_cat(mod_313, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.023',
 'precision: 0.473',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    9,     0,     9,   931],
        [    3,     0,     1,  2938],
        [    4,     0,    66, 53607],
        [   15,     0,    44,  6889]], dtype=int64))

#### 300

In [38]:
mod_300 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id300.hdf5')
mod_300_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id300.hdf5')
mod_300_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id300.hdf5')
mod_300_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id300.hdf5')

In [39]:
evaluate_cat(mod_300, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     0, 53676,     1],
        [    0,     0,  6947,     1]], dtype=int64))

In [41]:
evaluate_cat(mod_300_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     0, 53676,     1],
        [    0,     0,  6947,     1]], dtype=int64))

In [43]:
evaluate_cat(mod_300_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     0, 53676,     1],
        [    0,     0,  6947,     1]], dtype=int64))

In [45]:
evaluate_cat(mod_300_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     0, 53676,     1],
        [    0,     0,  6947,     1]], dtype=int64))

#### 301

In [47]:
mod_301 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id301.hdf5')
mod_301_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id301.hdf5')
mod_301_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id301.hdf5')
mod_301_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id301.hdf5')

In [48]:
evaluate_cat(mod_301, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.883',
 'precision: 0.901',
 'recall: 0.897',
 'accuracy: 0.897',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    5,     0,    21,   923],
        [    0,     0,     7,  2935],
        [    0,     0, 50968,  2709],
        [    0,     0,    19,  6929]], dtype=int64))

In [50]:
evaluate_cat(mod_301_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.854',
 'precision: 0.890',
 'recall: 0.859',
 'accuracy: 0.859',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    5,     0,    22,   922],
        [    0,     0,    31,  2911],
        [    0,     0, 48463,  5214],
        [    1,     0,    25,  6922]], dtype=int64))

In [52]:
evaluate_cat(mod_301_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.854',
 'precision: 0.893',
 'recall: 0.859',
 'accuracy: 0.859',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    5,     0,    22,   922],
        [    0,     0,     7,  2935],
        [    0,     0, 48461,  5216],
        [    0,     0,    19,  6929]], dtype=int64))

In [54]:
evaluate_cat(mod_301_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.883',
 'precision: 0.901',
 'recall: 0.897',
 'accuracy: 0.897',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    3,     0,    22,   924],
        [    0,     0,     7,  2935],
        [    0,     0, 50969,  2708],
        [    0,     0,    19,  6929]], dtype=int64))

#### 303

In [56]:
mod_303 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id303.hdf5')
mod_303_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id303.hdf5')
mod_303_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id303.hdf5')
mod_303_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id303.hdf5')

In [57]:
evaluate_cat(mod_303, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.901',
 'precision: 0.888',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,    55,   894],
        [    0,     0,   595,  2347],
        [    0,     0, 52731,   946],
        [    0,     0,    73,  6875]], dtype=int64))

In [59]:
evaluate_cat(mod_303_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.883',
 'precision: 0.885',
 'recall: 0.897',
 'accuracy: 0.897',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,    28,   921],
        [    0,     0,    97,  2845],
        [    0,     0, 50976,  2701],
        [    0,     0,    31,  6917]], dtype=int64))

In [61]:
evaluate_cat(mod_303_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.901',
 'precision: 0.889',
 'recall: 0.922',
 'accuracy: 0.922',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   11,     0,    55,   883],
        [   84,     0,   588,  2270],
        [   30,     0, 52735,   912],
        [  104,     0,    78,  6766]], dtype=int64))

In [63]:
evaluate_cat(mod_303_2, xbasic, ybasic)

('f1: 0.841',
 'precision: 0.938',
 'recall: 0.835',
 'accuracy: 0.835',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  128,   751,    55,    15],
        [  640,  2232,    67,     3],
        [   70,  2591, 50986,    30],
        [  691,  5620,    84,   553]], dtype=int64))

#### 304

In [65]:
mod_304 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id304.hdf5')
mod_304_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id304.hdf5')
mod_304_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id304.hdf5')
mod_304_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id304.hdf5')

In [66]:
evaluate_cat(mod_304, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.760',
 'precision: 0.781',
 'recall: 0.834',
 'accuracy: 0.834',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   935,    14],
        [    0,     0,  2941,     1],
        [    0,     0, 53663,    14],
        [    0,     0,  6821,   127]], dtype=int64))

In [68]:
evaluate_cat(mod_304_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     0, 53676,     1],
        [    0,     0,  6947,     1]], dtype=int64))

In [70]:
evaluate_cat(mod_304_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.844',
 'precision: 0.821',
 'recall: 0.879',
 'accuracy: 0.879',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   609,   340],
        [    0,     0,  2715,   227],
        [    0,     0, 53259,   418],
        [    0,     0,  3519,  3429]], dtype=int64))

In [72]:
evaluate_cat(mod_304_2, xbasic, ybasic)

('f1: 0.175',
 'precision: 0.652',
 'recall: 0.136',
 'accuracy: 0.136',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  359,   379,   181,    30],
        [  145,  2105,   531,   161],
        [  460, 39651,  6256,  7310],
        [ 3816,  2011,  1065,    56]], dtype=int64))

### 000

In [74]:
mod_000 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id000.hdf5')
mod_000_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id000.hdf5')
mod_000_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id000.hdf5')
mod_000_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id000.hdf5')

In [75]:
evaluate_cat(mod_000, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [76]:
evaluate_cat(mod_000_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [77]:
evaluate_cat(mod_000_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [78]:
evaluate_cat(mod_000_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

### 001

In [83]:
mod_001 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id001.hdf5')
mod_001_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id001.hdf5')
mod_001_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id001.hdf5')
mod_001_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id001.hdf5')

In [84]:
evaluate_cat(mod_001, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.904',
 'precision: 0.909',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    3,     0,    28,   918],
        [    0,     0,    16,  2926],
        [    0,     0, 52721,   956],
        [    0,     0,    31,  6917]], dtype=int64))

In [85]:
evaluate_cat(mod_001_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.903',
 'precision: 0.907',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    3,     0,    32,   914],
        [    0,     0,   210,  2732],
        [    0,     0, 52723,   954],
        [    0,     0,    32,  6916]], dtype=int64))

In [86]:
evaluate_cat(mod_001_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.904',
 'precision: 0.906',
 'recall: 0.925',
 'accuracy: 0.925',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    7,     0,    30,   912],
        [    0,     0,   109,  2833],
        [    0,     0, 52723,   954],
        [    1,     0,    31,  6916]], dtype=int64))

In [87]:
evaluate_cat(mod_001_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.904',
 'precision: 0.909',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    3,     0,    31,   915],
        [    0,     0,    45,  2897],
        [    0,     0, 52723,   954],
        [    0,     0,    32,  6916]], dtype=int64))

### 002

In [92]:
mod_002 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id002.hdf5')
mod_002_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id002.hdf5')
mod_002_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id002.hdf5')
mod_002_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id002.hdf5')

In [93]:
evaluate_cat(mod_002, xbasic, ybasic)

('f1: 0.685',
 'precision: 0.904',
 'recall: 0.567',
 'accuracy: 0.567',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  379,    44,    15,   511],
        [ 1110,   461,    15,  1356],
        [ 7704, 14248, 31220,   505],
        [ 2350,   107,     1,  4490]], dtype=int64))

In [94]:
evaluate_cat(mod_002_0, xbasic, ybasic)

('f1: 0.683',
 'precision: 0.904',
 'recall: 0.562',
 'accuracy: 0.562',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  433,    42,    14,   460],
        [ 1321,   417,    23,  1181],
        [ 6757, 15226, 31220,   474],
        [ 2664,    94,     1,  4189]], dtype=int64))

In [95]:
evaluate_cat(mod_002_1, xbasic, ybasic)

('f1: 0.681',
 'precision: 0.904',
 'recall: 0.560',
 'accuracy: 0.560',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  454,    42,    11,   442],
        [ 1326,   454,    14,  1148],
        [ 6776, 15227, 31218,   456],
        [ 2857,   102,     1,  3988]], dtype=int64))

In [96]:
evaluate_cat(mod_002_2, xbasic, ybasic)

('f1: 0.684',
 'precision: 0.904',
 'recall: 0.565',
 'accuracy: 0.565',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  394,    44,    14,   497],
        [ 1160,   447,    10,  1325],
        [ 7713, 14246, 31219,   499],
        [ 2422,   108,     1,  4417]], dtype=int64))

### 003

In [101]:
mod_003 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id003.hdf5')
mod_003_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id003.hdf5')
mod_003_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id003.hdf5')
mod_003_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id003.hdf5')

In [102]:
evaluate_cat(mod_003, xbasic, ybasic)

('f1: 0.802',
 'precision: 0.932',
 'recall: 0.796',
 'accuracy: 0.796',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    3,   907,    39,     0],
        [    0,  2856,    86,     0],
        [    3,  5173, 48500,     1],
        [  410,  6337,   192,     9]], dtype=int64))

In [103]:
evaluate_cat(mod_003_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.902',
 'precision: 0.890',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,    36,   913],
        [    0,     0,   487,  2455],
        [    0,     0, 52725,   952],
        [    0,     0,    32,  6916]], dtype=int64))

In [104]:
evaluate_cat(mod_003_1, xbasic, ybasic)

('f1: 0.884',
 'precision: 0.891',
 'recall: 0.885',
 'accuracy: 0.885',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[   27,   380,   104,   438],
        [  114,  1242,  1090,   496],
        [   30,   285, 52811,   551],
        [  203,  3271,   474,  3000]], dtype=int64))

In [105]:
evaluate_cat(mod_003_2, xbasic, ybasic)

('f1: 0.802',
 'precision: 0.932',
 'recall: 0.796',
 'accuracy: 0.796',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    3,   906,    40,     0],
        [    0,  2856,    86,     0],
        [    3,  5173, 48500,     1],
        [  404,  6334,   201,     9]], dtype=int64))

### 004

In [110]:
mod_004 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id004.hdf5')
mod_004_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id004.hdf5')
mod_004_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id004.hdf5')
mod_004_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id004.hdf5')

In [111]:
evaluate_cat(mod_004, xbasic, ybasic)

('f1: 0.756',
 'precision: 0.721',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    1,     0,   947,     1],
        [    0,     0,  2941,     1],
        [    1,     1, 53674,     1],
        [    8,     2,  6937,     1]], dtype=int64))

In [112]:
evaluate_cat(mod_004_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.728',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {1, 2, 3}',
 array([[    0,     0,   949,     0],
        [    0,     0,  2941,     1],
        [    0,     1, 53675,     1],
        [    0,     0,  6947,     1]], dtype=int64))

In [113]:
evaluate_cat(mod_004_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.757',
 'precision: 0.754',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[    0,     0,   943,     6],
        [    0,     0,  2939,     3],
        [    0,     0, 53660,    17],
        [    0,     0,  6913,    35]], dtype=int64))

In [114]:
evaluate_cat(mod_004_2, xbasic, ybasic)

('f1: 0.756',
 'precision: 0.730',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    1,     0,   948,     0],
        [    0,     0,  2941,     1],
        [    1,     1, 53674,     1],
        [    5,     2,  6940,     1]], dtype=int64))

### 010

In [119]:
mod_010 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id010.hdf5')
mod_010_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id010.hdf5')
mod_010_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id010.hdf5')
mod_010_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id010.hdf5')

In [120]:
evaluate_cat(mod_010, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.052',
 'precision: 0.461',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   23,     0,     6,   920],
        [  477,     0,     2,  2463],
        [41953,     0,    19, 11705],
        [   17,     0,    10,  6921]], dtype=int64))

In [121]:
evaluate_cat(mod_010_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.053',
 'precision: 0.374',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   27,     0,     9,   913],
        [  498,     0,     4,  2440],
        [41960,     0,    49, 11668],
        [   23,     0,    58,  6867]], dtype=int64))

In [122]:
evaluate_cat(mod_010_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.059',
 'precision: 0.456',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   30,     0,     6,   913],
        [  544,     0,     0,  2398],
        [45434,     0,    28,  8215],
        [   20,     0,    22,  6906]], dtype=int64))

In [123]:
evaluate_cat(mod_010_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.059',
 'precision: 0.499',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   26,     0,     6,   917],
        [  525,     0,     0,  2417],
        [45429,     0,    16,  8232],
        [   19,     0,     7,  6922]], dtype=int64))

### 011

In [128]:
mod_011 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/id011.hdf5')
mod_011_0 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id011.hdf5')
mod_011_1 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id011.hdf5')
mod_011_2 = tf.keras.models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id011.hdf5')

In [129]:
evaluate_cat(mod_011, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.692',
 'precision: 0.788',
 'recall: 0.649',
 'accuracy: 0.649',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   22,     0,   400,   527],
        [    0,     0,   165,  2777],
        [   15,     0, 37673, 15989],
        [   40,     0,  2757,  4151]], dtype=int64))

In [130]:
evaluate_cat(mod_011_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.732',
 'precision: 0.793',
 'recall: 0.705',
 'accuracy: 0.705',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   16,     0,   431,   502],
        [    0,     0,   154,  2788],
        [   14,     0, 41469, 12194],
        [   30,     0,  2928,  3990]], dtype=int64))

In [131]:
evaluate_cat(mod_011_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.717',
 'precision: 0.783',
 'recall: 0.685',
 'accuracy: 0.685',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   24,     0,   452,   473],
        [    0,     0,   217,  2725],
        [   17,     0, 40503, 13157],
        [   47,     0,  3220,  3681]], dtype=int64))

In [132]:
evaluate_cat(mod_011_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.689',
 'precision: 0.781',
 'recall: 0.645',
 'accuracy: 0.645',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   25,     0,   432,   492],
        [    0,     0,   232,  2710],
        [   18,     0, 37703, 15956],
        [   51,     0,  2994,  3903]], dtype=int64))

#### 000, 001, 002, 003, 004, 010, 011, CAT500, CAT501, CAT502, CAT700, CAT701, CAT702

In [7]:
mod_000 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id000.hdf5')
mod_001 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id001.hdf5')
mod_002 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id002.hdf5')
mod_003 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id003.hdf5')
mod_004 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id004.hdf5')
mod_010 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id010.hdf5')
mod_011 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id011.hdf5')
cat500 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT500.hdf5')
cat501 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT501.hdf5')
cat502 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT502.hdf5')
cat700 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT700.hdf5')
cat701 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT701.hdf5')
cat702 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idCAT702.hdf5')

In [7]:
evaluate_cat(mod_000, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [8]:
evaluate_cat(mod_000, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.100',
 'precision: 0.062',
 'recall: 0.250',
 'accuracy: 0.250',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2}',
 array([[  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0]], dtype=int64))

In [9]:
evaluate_cat(mod_001, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.904',
 'precision: 0.909',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[    3,     0,    28,   918],
        [    0,     0,    16,  2926],
        [    0,     0, 52721,   956],
        [    0,     0,    31,  6917]], dtype=int64))

In [10]:
evaluate_cat(mod_001, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.341',
 'precision: 0.268',
 'recall: 0.494',
 'accuracy: 0.494',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[  0,   0,  53, 896],
        [  0,   0, 372, 577],
        [  0,   0, 933,  16],
        [  0,   0,   5, 944]], dtype=int64))

In [11]:
evaluate_cat(mod_002, xcat, ycat)

('f1: 0.685',
 'precision: 0.904',
 'recall: 0.567',
 'accuracy: 0.567',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  379,    44,    15,   511],
        [ 1110,   461,    15,  1356],
        [ 7704, 14248, 31220,   505],
        [ 2350,   107,     1,  4490]], dtype=int64))

In [12]:
evaluate_cat(mod_002, xcat_eq, ycat_eq)

('f1: 0.475',
 'precision: 0.542',
 'recall: 0.525',
 'accuracy: 0.525',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[531,  35,  56, 327],
        [427,  67, 276, 179],
        [ 35,   0, 905,   9],
        [438,  13,   7, 491]], dtype=int64))

In [13]:
evaluate_cat(mod_003, xcat, ycat)

('f1: 0.802',
 'precision: 0.932',
 'recall: 0.796',
 'accuracy: 0.796',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    3,   907,    39,     0],
        [    0,  2856,    86,     0],
        [    3,  5173, 48500,     1],
        [  410,  6337,   192,     9]], dtype=int64))

In [14]:
evaluate_cat(mod_003, xcat_eq, ycat_eq)

('f1: 0.265',
 'precision: 0.515',
 'recall: 0.383',
 'accuracy: 0.383',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  3, 794, 152,   0],
        [  0, 513, 436,   0],
        [  0,  13, 936,   0],
        [  9, 815, 124,   1]], dtype=int64))

In [15]:
evaluate_cat(mod_004, xcat, ycat)

('f1: 0.756',
 'precision: 0.721',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[    1,     0,   947,     1],
        [    0,     0,  2941,     1],
        [    1,     1, 53674,     1],
        [    8,     2,  6937,     1]], dtype=int64))

In [16]:
evaluate_cat(mod_004, xcat_eq, ycat_eq)

('f1: 0.171',
 'precision: 0.196',
 'recall: 0.242',
 'accuracy: 0.242',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[142, 462, 344,   1],
        [  3, 765, 181,   0],
        [  1, 938,  10,   0],
        [158, 317, 474,   0]], dtype=int64))

In [17]:
evaluate_cat(mod_010, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.052',
 'precision: 0.461',
 'recall: 0.108',
 'accuracy: 0.108',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   23,     0,     6,   920],
        [  477,     0,     2,  2463],
        [41953,     0,    19, 11705],
        [   17,     0,    10,  6921]], dtype=int64))

In [18]:
evaluate_cat(mod_010, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.358',
 'precision: 0.333',
 'recall: 0.496',
 'accuracy: 0.496',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[ 10,   0,  45, 894],
        [ 40,   0, 205, 704],
        [  2,   0, 933,  14],
        [  5,   0,   3, 941]], dtype=int64))

In [19]:
evaluate_cat(mod_011, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.692',
 'precision: 0.788',
 'recall: 0.649',
 'accuracy: 0.649',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[   22,     0,   400,   527],
        [    0,     0,   165,  2777],
        [   15,     0, 37673, 15989],
        [   40,     0,  2757,  4151]], dtype=int64))

In [20]:
evaluate_cat(mod_011, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.343',
 'precision: 0.518',
 'recall: 0.495',
 'accuracy: 0.495',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  3,   0,  55, 891],
        [  0,   0, 369, 580],
        [  0,   0, 933,  16],
        [  0,   0,   5, 944]], dtype=int64))

In [8]:
evaluate_cat(cat500, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [9]:
evaluate_cat(cat500, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.100',
 'precision: 0.062',
 'recall: 0.250',
 'accuracy: 0.250',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2}',
 array([[  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0]], dtype=int64))

In [10]:
evaluate_cat(cat501, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.820',
 'precision: 0.900',
 'recall: 0.774',
 'accuracy: 0.774',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[  740,     0,    24,   185],
        [ 2204,     0,   234,   504],
        [ 6031,     0, 47493,   153],
        [ 5189,     0,    26,  1733]], dtype=int64))

In [11]:
evaluate_cat(cat501, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.437',
 'precision: 0.382',
 'recall: 0.513',
 'accuracy: 0.513',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2, 3}',
 array([[510,   0,  62, 377],
        [360,   0, 322, 267],
        [ 10,   0, 934,   5],
        [440,   0,   7, 502]], dtype=int64))

In [12]:
evaluate_cat(cat502, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.900',
 'precision: 0.899',
 'recall: 0.924',
 'accuracy: 0.924',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {1, 2, 3}',
 array([[    0,     0,    55,   894],
        [    0,     1,  1095,  1846],
        [    0,     1, 52743,   933],
        [    0,     1,    50,  6897]], dtype=int64))

In [13]:
evaluate_cat(cat502, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.337',
 'precision: 0.262',
 'recall: 0.491',
 'accuracy: 0.491',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2, 3}',
 array([[  0,   0,  96, 853],
        [  0,   0, 374, 575],
        [  0,   0, 936,  13],
        [  0,   0,  20, 929]], dtype=int64))

In [14]:
evaluate_cat(cat700, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.756',
 'precision: 0.692',
 'recall: 0.832',
 'accuracy: 0.832',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 2}',
 array([[    0,     0,   949,     0],
        [    1,     0,  2941,     0],
        [    1,     0, 53676,     0],
        [    1,     0,  6947,     0]], dtype=int64))

In [15]:
evaluate_cat(cat700, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.100',
 'precision: 0.062',
 'recall: 0.250',
 'accuracy: 0.250',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {2}',
 array([[  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0],
        [  0,   0, 949,   0]], dtype=int64))

In [16]:
evaluate_cat(cat701, xcat, ycat)

('f1: 0.388',
 'precision: 0.892',
 'recall: 0.302',
 'accuracy: 0.302',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[  166,    16,     3,   764],
        [  503,   624,     2,  1813],
        [ 5717, 33088, 12363,  2509],
        [  608,    19,     2,  6319]], dtype=int64))

In [17]:
evaluate_cat(cat701, xcat_eq, ycat_eq)

('f1: 0.545',
 'precision: 0.647',
 'recall: 0.575',
 'accuracy: 0.575',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[391,   8,  48, 502],
        [298, 205, 253, 193],
        [ 35,   0, 905,   9],
        [258,   3,   6, 682]], dtype=int64))

In [18]:
evaluate_cat(cat702, xcat, ycat)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.896',
 'precision: 0.932',
 'recall: 0.912',
 'accuracy: 0.912',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {1, 2, 3}',
 array([[    0,     3,    40,   906],
        [    0,   149,   252,  2541],
        [    0,     1, 51763,  1913],
        [    0,     0,    52,  6896]], dtype=int64))

In [19]:
evaluate_cat(cat702, xcat_eq, ycat_eq)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.368',
 'precision: 0.440',
 'recall: 0.504',
 'accuracy: 0.504',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {1, 2, 3}',
 array([[  0,  17,  61, 871],
        [  0,  48, 324, 577],
        [  0,   2, 934,  13],
        [  0,   5,  11, 933]], dtype=int64))